# Analyze sessions in batch from Phase 1 of AdaDrive (work in progress)

In [1]:
import sys
 
# setting path
sys.path.append('..')

from mna.utils.rnapp_data_format import read_all_lslpresets, return_metadata_from_name, event_data_from_data
import pickle, os
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display

import matplotlib.pyplot as plt
from mna.sessions.eye_session import process_session_eye
from mna.sessions.eeg_session import process_session_eeg
from mna.sessions.motor_session import process_session_motor
from mna.sessions.ecg_session import process_session_ecg
from os import listdir
from os.path import isfile, join
from mna.utils.rnapp_data_format import read_all_lslpresets, return_metadata_from_name, event_data_from_data
import pickle
from statannotations.Annotator import Annotator
from collections import defaultdict
from scipy import stats
import mne
import glob 
import random
import math
import statsmodels.api as sm

# 1. Read a RN App, converted pkl file, and create the metadata and data structure

In [2]:
import matplotlib
matplotlib.use('Agg')

# Prep

In [3]:
from mna.utils.rnapp_data_format import read_all_files

In [75]:
data_dir = "../data/"
lsl_dir = "../mna/LSLPresets/"
output_dir = '../output/exposure/'
if not os.path.isdir(output_dir): os.makedirs(output_dir)
metadata_jsons = read_all_lslpresets(path_to_jsonfiles=lsl_dir)
onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and '.pkl' in f]

remove_sessions = [(15,1),(22,1)]
reference_ica = "sbj20ssn03"
save_data_pkl = True # save data into pickle files
save_ica_plts = False # save ICA components plots
epoch_raw_eeg = False # epoching raw data
motor_events = True
preturn = 1000
rs = 64 # random seed

# Read Annotated Data

In [76]:
# motor_events = pd.read_excel(f"{data_dir}/annotated/all_motor_break_events.xlsx")
# motor_events

In [6]:
import os
motor_events = []
for file in os.listdir(f"{data_dir}/annotated/"):
    if file.endswith("_trial_df.csv"):
        motor_events.append(pd.read_csv(f"{data_dir}/annotated/{file}"))
motor_events = pd.concat(motor_events)
motor_events['end_intervals'] = motor_events.timestamps
motor_events['start_intervals'] = motor_events.timestamps.shift(1)
motor_events = motor_events[motor_events.block_condition=='exposure']
motor_events['sbj_ssn'] = motor_events.apply(lambda x: f"sbj{'%02d' % (x['ppid'],)}ssn{'%02d' % (x['session'],)}",axis=1)

In [7]:
# get the exposure periods of interest
exposure_periods = defaultdict(list)
exposure_periods_sub = defaultdict(list)
for sub in motor_events.sbj_ssn.unique():
    start_intervals = []
    found_calib = False
    sub_df = motor_events.loc[motor_events.sbj_ssn==sub].reset_index(drop=True)
    sbj_ssn = sub_df.sbj_ssn.iloc[0]
    for index, period in sub_df.iterrows():
        end_interval = period.end_intervals
        start_interval = period.start_intervals
        opacity = period.density
        exposure_periods[sbj_ssn].append((start_interval, end_interval, opacity))

# link the ids with the file
for each_file in onlyfiles:
    input_path = data_dir + each_file
    
    sbj_id = each_file[each_file.find('Sbj_')+4:each_file.find('-Ssn')]
    ssn_no = each_file[each_file.find('Ssn_')+4:each_file.find('.dats')]

    if len(sbj_id) < 2: sbj = "sbj0"+sbj_id
    else: sbj = "sbj"+sbj_id
    if len(ssn_no) < 2: ssn = "ssn0"+ssn_no
    else: ssn = "ssn"+ssn_no
    if sbj+ssn in exposure_periods:
        exposure_periods[sbj+ssn].append(sbj+ssn)
        exposure_periods[sbj+ssn].append(input_path)
    exposure_periods_sub[sbj].append(exposure_periods[sbj+ssn])

## EEG Exports

In [79]:
eeg_montage = 'biosemi64'
eeg_channel = 'BioSemi'

In [80]:
all_epochs = []
for pp in exposure_periods_sub:
    print('processing pp', pp)
    for session in exposure_periods_sub[pp]:
        input_path = session[-1]
        pid = int(session[-2].split('sbj')[1].split('ssn')[0])
        sess = int(session[-2].split('sbj')[1].split('ssn')[1])
        skip = False
        for es in remove_sessions:
            if pid==es[0] and sess==es[1]:
                print(f"skipping {pid} {sess}")
                skip = True
        if skip: continue
        print('processing session', input_path)
        with open(input_path, 'rb') as handle:
            rns_data = pickle.load(handle)

        for key in rns_data.keys():
            rns_data[key].append(return_metadata_from_name(key, metadata_jsons))

        eeg_channel_names = mne.channels.make_standard_montage(eeg_montage).ch_names
        df = pd.DataFrame(rns_data[eeg_channel][0], columns=rns_data[eeg_channel][1],
                          index=rns_data[eeg_channel][2]['ChannelNames']).T
        starting_time_s = rns_data[eeg_channel][1][0]
        freq = rns_data[eeg_channel][2]['NominalSamplingRate']
        rna_channel_names = list(df.columns)
        rna_channel_names[1:65] = eeg_channel_names
        info = mne.create_info(ch_names=rna_channel_names, ch_types=['stim'] + ['eeg'] * 64 + ['ecg'] * 2 + ['misc'] * 22,
                               sfreq=freq)
        info.set_montage(eeg_montage)

        raw = mne.io.RawArray(df.T * 1e-6, info)
        raw = raw.pick('eeg')
        raw.resample(freq/16) # resample to 128 Hz
        freq /= 16
        trial_start_time = np.array(session[:-2])[:,0] - starting_time_s  # reference for mne
        opacities = np.array(session[:-2])[:,2]
        events = np.column_stack((trial_start_time * freq,
                                  np.zeros(len(trial_start_time), dtype=int),
                                  opacities)).astype(int)
        metadata_df = pd.DataFrame(session[:-2])
        metadata_df = metadata_df.rename(columns={0:'start_time_lsl', 1: 'end_time_lsl', 2: 'opacity'})
        metadata_df['pid'] = pid
        metadata_df['sess'] = sess
        epochs = mne.Epochs(raw, events, tmin=0, tmax=5, baseline=None, preload=True, 
                        on_missing='warn', metadata=metadata_df)
        all_epochs.append(epochs)

processing pp sbj20
processing session ../data/09_14_2022_13_17_39-Exp_adadrive-Sbj_20-Ssn_3.dats.pkl
Creating RawArray with float64 data, n_channels=89, n_times=3399947
    Range : 0 ... 3399946 =      0.000 ...  1660.130 secs
Ready.
Adding metadata with 5 columns
20 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20 events and 641 original time points ...
0 bad epochs dropped
processing session ../data/09_13_2022_13_38_39-Exp_adadrive-Sbj_20-Ssn_2.dats.pkl
Creating RawArray with float64 data, n_channels=89, n_times=2643232
    Range : 0 ... 2643231 =      0.000 ...  1290.640 secs
Ready.
Adding metadata with 5 columns
20 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20 events and 641 original time points ...
0 bad epochs dropped
processing session ../data/09_08_2022_13_28_01-Exp_adadrive-Sbj_20-Ssn_1.dats.pkl
Creating RawArray with float64 data, n_c

In [81]:
with open(f"{output_dir}exposure_epochs.pickle", 'wb') as handle_eps:
    pickle.dump(all_epochs, handle_eps, protocol=pickle.HIGHEST_PROTOCOL)

In [82]:
print('done')

done


## Eye data exports and fitting

In [8]:
channel = 'L Pupil Diameter'
freq = 120

In [9]:
all_dfs = []
agg_dfs = []
for pp in exposure_periods_sub:
    print('processing pp', pp)
    participant_df = []
    for session in exposure_periods_sub[pp]:
        input_path = session[-1]
        print('processing session', input_path)
        with open(input_path, 'rb') as handle:
            rns_data = pickle.load(handle)

        for key in rns_data.keys():
            rns_data[key].append(return_metadata_from_name(key, metadata_jsons))

        eye_channel='Unity_ViveSREyeTracking'
        if eye_channel in rns_data:
            df_eye = pd.DataFrame(rns_data[eye_channel][0], columns=rns_data[eye_channel][1],
                                  index=rns_data[eye_channel][2]['ChannelNames']).T
            df_eye = df_eye.reset_index().rename(columns={'index':'timestamp'})
            for calib_period_index,period in enumerate(session[:-2]):
                sub_df = df_eye.loc[(df_eye.timestamp >= period[0]) & (df_eye.timestamp <= period[1]), ['timestamp', channel]].reset_index(drop=True).reset_index()
                sub_df = sub_df.rename(columns={'index': 'ref_timestamp'})
                sub_df['sub_sess'] = session[-2]
                sub_df['calib_period_index'] = calib_period_index
                sub_df['opacities'] = period[2]

                sub_df[channel+' Transform'] = sub_df[channel].replace(-1, np.nan)

                # get first found pupil and baseline adjust
                starting_diameter_idx = sub_df[channel].notna().idxmax()
                starting_diameter = sub_df[channel].iloc[starting_diameter_idx]
                sub_df[channel+' Transform'] = sub_df[channel+' Transform'] - starting_diameter

                sub_df[channel] = sub_df[channel].interpolate(method ='linear', limit_direction ='forward')
                sub_df[channel+' Transform'] = sub_df[channel+' Transform'].rolling(30).mean() # 250 ms meaning average filter
                participant_df.append(sub_df)
    participant_df = pd.concat(participant_df)
    # fit opacities to pupil diameter
    sub_df_nonna = participant_df[['L Pupil Diameter Transform','opacities']].dropna()
    Y = sub_df_nonna['L Pupil Diameter Transform']
    X = sub_df_nonna['opacities']
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    participant_df['w_opacities'] = results.params['opacities']
    participant_df['const'] = results.params['const']
    participant_df['p_opacities'] = results.pvalues['opacities']
    
    participant_df[['sub', 'sess']] = participant_df['sub_sess'].str.split('sbj', 1, expand=True)[1].str.split('ssn', 1, expand=True)
    participant_df['sub'] = participant_df['sub'].astype(int)
    participant_df['sess'] = participant_df['sess'].astype(int)
    agg_df = participant_df.groupby(['sub']).mean()[['L Pupil Diameter','w_opacities','const','p_opacities']]
    all_dfs.append(agg_df)
    agg_dfs.append(agg_df)
agg_dfs = pd.concat(agg_dfs)


processing pp sbj20
processing session ../data/09_14_2022_13_17_39-Exp_adadrive-Sbj_20-Ssn_3.dats.pkl
processing session ../data/09_13_2022_13_38_39-Exp_adadrive-Sbj_20-Ssn_2.dats.pkl
processing session ../data/09_08_2022_13_28_01-Exp_adadrive-Sbj_20-Ssn_1.dats.pkl
processing pp sbj14
processing session ../data/09_13_2022_16_22_31-Exp_adadrive-Sbj_14-Ssn_3.dats.pkl
processing session ../data/09_05_2022_15_44_37-Exp_adadrive-Sbj_14-Ssn_01.dats.pkl
processing session ../data/09_07_2022_16_41_37-Exp_adadrive-Sbj_14-Ssn_2.dats.pkl
processing pp sbj12
processing session ../data/09_02_2022_10_34_48-Exp_adadrive-Sbj_12-Ssn_03.dats.pkl
processing session ../data/08_25_2022_10_32_35-Exp_adadrive-Sbj_12-Ssn_01.dats.pkl
processing session ../data/08_26_2022_11_53_52-Exp_adadrive-Sbj_12-Ssn_02.dats.pkl
processing pp sbj22
processing session ../data/09_15_2022_15_31_24-Exp_adadrive-Sbj_22-Ssn_1.dats.pkl
processing pp sbj16
processing session ../data/09_21_2022_14_26_45-Exp_adadrive-Sbj_16-Ssn_03.da

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

# Summary Stats

In [16]:
agg_dfs.to_csv(f"{output_dir}participant_level_exposure_fits.csv")

In [14]:
all_dfs

,L Pupil Diameter,w_opacities,const,p_opacities
sub,,,,
20,2.109190,-0.634344,0.803078,2.962203e-87
14,2.185119,-0.152842,0.239942,2.603150e-15
12,1.795109,-1.059623,0.963646,0.000000e+00
22,2.497604,-1.177283,1.328638,1.513329e-43
16,2.949962,-0.272411,0.371201,6.657299e-37
18,2.274272,-1.228944,0.982158,5.331002e-298
19,2.462633,0.336097,-0.126547,4.757658e-128
21,2.079794,-0.436888,0.459473,1.793245e-102
15,2.403911,-3.060702,2.153450,0.000000e+00


# Plots

## Pupil vs. opacity by pp and session

In [34]:
%matplotlib inline
sns.lmplot(
    data=all_dfs, x="opacities", y=channel+' Transform', col='sub', row='sub_sess',
    palette="muted", ci=None, 
    height=4, scatter_kws={"s": 10, "alpha": .6}
)
plt.savefig(f"{output_dir}left_pupil_v_opacity.png", dpi=300)

## Baseline corrected pupil vs. time during exposure period

In [ ]:
%matplotlib inline

sns.lmplot(
    data=all_dfs, x="ref_timestamp", y=channel+' Transform', col="calib_period_index", hue="opacities", row='sub_sess',
    palette="muted", ci=None, fit_reg=False,
    height=4, scatter_kws={"s": 10, "alpha": .6}
)
plt.savefig(f"{output_dir}left_pupil_v_time_by_opacity.png", dpi=300)

In [30]:
all_dfs.sub_sess.unique()

array(['sbj19ssn03', 'sbj13ssn03', 'sbj23ssn02', 'sbj21ssn03',
       'sbj12ssn03', 'sbj20ssn01', 'sbj20ssn03', 'sbj16ssn01',
       'sbj16ssn03', 'sbj21ssn01', 'sbj18ssn02', 'sbj19ssn01',
       'sbj15ssn01', 'sbj14ssn03', 'sbj18ssn03', 'sbj19ssn02',
       'sbj14ssn01', 'sbj12ssn01', 'sbj23ssn01', 'sbj20ssn02',
       'sbj17ssn03', 'sbj21ssn02', 'sbj12ssn02', 'sbj14ssn02',
       'sbj13ssn01', 'sbj18ssn01', 'sbj23ssn03', 'sbj16ssn02',
       'sbj13ssn02', 'sbj22ssn01'], dtype=object)